In [1]:
import sys
import os
import struct
from google.protobuf.json_format import MessageToJson
import json
import numpy as np

sys.path.insert(1, 'lri-proto/python_bindings')

# Enumerate your BlockTypes in Python to match the Rust definition
class BlockType:
    LightHeader = 0
    ViewPreferences = 1
    GPSData = 2
    # Add other types as needed

def read_binary_file(filepath):
    with open(filepath, "rb") as file:
        content = file.read()
    return content


def parse_header_mod(data):
    magic = b"LELR"
    if data[:4] != magic:
        raise ValueError("Magic number is incorrect")

    combined_length, message_offset, message_length = struct.unpack('<QQL', data[4:24])
    kind = data[24]

    if kind == BlockType.LightHeader:
        message_type = "lightheader_pb2.LightHeader"
    elif kind == BlockType.ViewPreferences:
        message_type = "view_preferences_pb2.ViewPreferences"
    elif kind == BlockType.GPSData:
        message_type = "gps_data_pb2.GPSData"
    else:
        raise ValueError(f"Unknown block type {kind}")

    return combined_length, message_offset, message_length, message_type
    

from view_preferences_pb2 import ViewPreferences
from gps_data_pb2 import GPSData
from lightheader_pb2 import LightHeader

In [2]:
file_path = "../goose.lri"
binary_content = read_binary_file(file_path)
binary_content_temp = binary_content
previous = 0 
block_info_json = []
json_light_header = []
block_info = []
while len(binary_content_temp)>0:
    header_info = (parse_header_mod(binary_content_temp))
#     header_info.append)
    if(header_info[3]=="lightheader_pb2.LightHeader"):
        light_header = LightHeader()
        light_header.ParseFromString(binary_content_temp[header_info[1]:header_info[1]+header_info[2]])
        
        
    if(header_info[3]=="view_preferences_pb2.ViewPreferences"):
        light_header = ViewPreferences()
        light_header.ParseFromString(binary_content_temp[header_info[1]:header_info[1]+header_info[2]])
        
    if(header_info[3]=="gps_data_pb2.GPSData"):
        light_header = GPSData()
        light_header.ParseFromString(binary_content_temp[header_info[1]:header_info[1]+header_info[2]])

    light_header_json = MessageToJson(light_header)
    json_light_header.append(json.loads(light_header_json))
    
    block_info.append(binary_content_temp[:header_info[0]])
    binary_content_temp=binary_content_temp[header_info[0]:]

In [3]:
focus = json_light_header[0]["modules"][0]
focus

{'afInfo': {'mode': 'AUTO',
  'roiCenter': {'x': 0.43100962, 'y': 0.4701923},
  'disparityFocusDistance': 3863.0,
  'contrastFocusDistance': 21278.104,
  'lensTimeout': False,
  'mirrorTimeout': False,
  'mirrorPosition': 677},
 'id': 'B2',
 'isEnabled': True,
 'mirrorPosition': 678,
 'lensPosition': 1059,
 'sensorAnalogGain': 5.0,
 'sensorExposure': '511041',
 'sensorDataSurface': {'start': {'x': 0, 'y': 0},
  'size': {'x': 4160, 'y': 3120},
  'format': 'RAW_PACKED_10BPP',
  'rowStride': 5200,
  'dataOffset': '32',
  'dataScale': {'x': 1.0, 'y': 1.0}},
 'sensorTemparature': 34,
 'sensorBayerRedOverride': {'x': 1, 'y': 1},
 'sensorDigitalGain': 1.03125,
 'frameIndex': 0,
 'sensorDpcOn': True}

In [4]:
perFocusCalibration = [i for i in json_light_header[4]["moduleCalibration"] if i["cameraId"]=="B2"]

In [5]:
vignetting = [i for i in json_light_header[5]["moduleCalibration"] if i["cameraId"]=="B2"][0].keys()

In [6]:
color = [i for i in json_light_header[7]["moduleCalibration"] if i["cameraId"]=="B2"][0]["color"][0]

In [53]:
color_a = [i for i in json_light_header[7]["moduleCalibration"] if i["cameraId"]=="B2"][1]["color"][0]

In [57]:
color_f11 = [i for i in json_light_header[7]["moduleCalibration"] if i["cameraId"]=="B2"][2]["color"][0]
color_f11

{'type': 'F11',
 'forwardMatrix': {'x00': 0.7936,
  'x01': 0.08120257,
  'x02': 0.08941732,
  'x10': 0.21110979,
  'x11': 0.9459514,
  'x12': -0.15706119,
  'x20': -0.113330044,
  'x21': -0.47036007,
  'x22': 1.4089},
 'colorMatrix': {'x00': 0.68549967,
  'x01': -0.10727772,
  'x02': -0.0407746,
  'x10': -0.31762183,
  'x11': 1.1033603,
  'x12': 0.24587315,
  'x20': -0.026844714,
  'x21': 0.21904309,
  'x22': 0.48594573},
 'rgRatio': 0.5872502,
 'bgRatio': 0.53582305,
 'macbethData': [{'x': 0.07652555, 'y': 0.09140972, 'z': 0.04152661},
  {'x': 0.26528546, 'y': 0.3199688, 'z': 0.15131825},
  {'x': 0.097522855, 'y': 0.20355529, 'z': 0.14228617},
  {'x': 0.06810701, 'y': 0.13808864, 'z': 0.04997688},
  {'x': 0.14306688, 'y': 0.24636804, 'z': 0.17975886},
  {'x': 0.17548466, 'y': 0.44941399, 'z': 0.22551808},
  {'x': 0.278227, 'y': 0.24140549, 'z': 0.081786625},
  {'x': 0.06450694, 'y': 0.13821654, 'z': 0.13788208},
  {'x': 0.22028054, 'y': 0.16319601, 'z': 0.08401432},
  {'x': 0.04736609

In [7]:
color.keys()

dict_keys(['type', 'forwardMatrix', 'colorMatrix', 'rgRatio', 'bgRatio', 'macbethData', 'spectralData'])

In [41]:
color["type"]

'D65'

In [42]:
color2 = [i for i in json_light_header[8]["moduleCalibration"] if i["cameraId"]=="B2"][0]["color"][0]

KeyError: 'moduleCalibration'

In [48]:
 json_light_header[5]#["moduleCalibration"] 

{'moduleCalibration': [{'cameraId': 'A1',
   'vignetting': {'crosstalk': {'width': 17,
     'height': 13,
     'dataPacked': [1.0457642,
      0.010062512,
      0.010062512,
      0.0,
      -0.029859742,
      0.9953632,
      0.0,
      -0.0021962195,
      -0.003938703,
      0.0,
      0.9916371,
      -0.0070909956,
      0.0,
      -0.0003788775,
      -0.0003788775,
      0.990475,
      1.0316406,
      0.014259291,
      0.014259291,
      0.0,
      -0.011352252,
      0.98124135,
      0.0,
      0.018722136,
      -0.012980769,
      0.0,
      0.98476756,
      0.01234993,
      0.0,
      0.0069783526,
      0.0069783526,
      0.9495998,
      1.0387986,
      0.012187232,
      0.012187232,
      0.0,
      -0.0062524504,
      0.9802773,
      0.0,
      0.016244842,
      -0.016453583,
      0.0,
      0.9891143,
      0.009542882,
      0.0,
      0.0059509287,
      0.0059509287,
      0.95441437,
      1.0330535,
      0.013054044,
      0.013054044,
      0.0,
  

In [8]:
colorMatrix_dict = color["colorMatrix"]
forwardMatrix_dict = color["forwardMatrix"]
colorMatrix_dict,forwardMatrix_dict

({'x00': 0.7232608,
  'x01': -0.12051902,
  'x02': -0.034705322,
  'x10': -0.34320852,
  'x11': 1.1315529,
  'x12': 0.24160586,
  'x20': -0.048811756,
  'x21': 0.2596398,
  'x22': 0.4647175},
 {'x00': 0.8681334,
  'x01': 0.15782288,
  'x02': -0.061736338,
  'x10': 0.29942137,
  'x11': 1.0843414,
  'x12': -0.3837628,
  'x20': -0.04755058,
  'x21': -0.4467093,
  'x22': 1.3194698})

In [9]:
li = []
li2 = []
for i in range(3):
    l = []
    l2 = []
    for j in range(3):
        l.append(colorMatrix_dict[f"x{i}{j}"])
        l2.append(forwardMatrix_dict[f"x{i}{j}"])
    li.append(l)
    li2.append(l2)

In [10]:
color_matrix = np.array(li)
color_matrix

array([[ 0.7232608 , -0.12051902, -0.03470532],
       [-0.34320852,  1.1315529 ,  0.24160586],
       [-0.04881176,  0.2596398 ,  0.4647175 ]])

In [11]:
forward_matrix = np.array(li2)
forward_matrix

array([[ 0.8681334 ,  0.15782288, -0.06173634],
       [ 0.29942137,  1.0843414 , -0.3837628 ],
       [-0.04755058, -0.4467093 ,  1.3194698 ]])

In [12]:
sensorDataSurface = focus["sensorDataSurface"]
sensorDataSurface

{'start': {'x': 0, 'y': 0},
 'size': {'x': 4160, 'y': 3120},
 'format': 'RAW_PACKED_10BPP',
 'rowStride': 5200,
 'dataOffset': '32',
 'dataScale': {'x': 1.0, 'y': 1.0}}

In [13]:
block_focus = block_info[0]

In [14]:
start_offset = int(focus["sensorDataSurface"]["dataOffset"])

final_offset = start_offset + sensorDataSurface["rowStride"]*sensorDataSurface["size"]["y"]

final_offset

16224032

In [15]:
raw_data = block_focus[start_offset:final_offset]

In [16]:
import math

# Define the TEN_MASK constant
TEN_MASK = 1023  # ten bits

def tenbit(packd, count, upack):
    required_len_packd = math.ceil(count * (10.0 / 8.0))

    if count > len(upack):
        raise ValueError(f"expected output buffer to be {count} bytes, got {len(upack)} bytes")

    if required_len_packd > len(packd):
        raise ValueError(f"expected input to be at least {required_len_packd} bytes, it was {len(packd)} bytes")

    packd = list(packd[:required_len_packd])[::-1]  # Reverse the list
    chunker = [packd[i:i + 5] for i in range(0, len(packd), 5)]  # Create chunks of size 5
    remain = packd[len(chunker) * 5:]  # Get the remainder

    for idx, chnk in enumerate(chunker):
        # Convert chunk to a long integer
        long = int.from_bytes(chnk + [0, 0, 0], byteorder='big')

        b4 = long & TEN_MASK
        b3 = (long >> 10) & TEN_MASK
        b2 = (long >> 20) & TEN_MASK
        b1 = (long >> 30) & TEN_MASK

        base_idx = idx * 4
        upack[base_idx:base_idx + 4] = [b1, b2, b3, b4]

    if remain:
        long_bytes = remain + [0] * (8 - len(remain))
        long = int.from_bytes(long_bytes, byteorder='little')

        count_remain = count % 4
        start = count - count_remain
        for idx in range(count_remain):
            upack[start + idx] = ((long >> (10 * idx)) & TEN_MASK)
    return upack

In [17]:
width = 4160  # Example width
height = 3120  # Example height

count = width * height;
count

12979200

In [18]:
upack = np.zeros(count)

In [19]:
rei = tenbit(raw_data, count, upack)

In [20]:
image_new = np.array(rei).reshape(height,width)
image_new.shape

(3120, 4160)

In [76]:
image_new[:8,:8]

array([[  516.,   208.,     0.,     0.,   726.,   960.,     0.,     0.],
       [  594.,   240.,     0.,     0.,   819.,   448.,     0.,     0.],
       [  421.,   272.,     0.,     0.,   935.,   592.,     0.,     0.],
       [  995.,   144.,     0.,     0.,   259.,   672.,     0.,     0.],
       [  502.,    48.,     0.,     0.,   440.,    48.,     0.,     0.],
       [  275.,   272.,     0.,     0.,   259.,    64.,     0.,     0.],
       [  504.,   832.,     0.,     0.,  1016.,    16.,     0.,     0.],
       [  803.,   880.,     0.,     0.,   755.,   256.,     0.,     0.]])

In [60]:
normalized_image = image_new /1023

In [61]:
color.keys()

dict_keys(['type', 'forwardMatrix', 'colorMatrix', 'rgRatio', 'bgRatio', 'macbethData', 'spectralData'])

In [62]:
M_camera_calibration_1 = np.identity(3)
M_camera_calibration_2 = np.identity(3)

In [25]:
from colour_demosaicing import (
    demosaicing_CFA_Bayer_bilinear,
    demosaicing_CFA_Bayer_Malvar2004,
    demosaicing_CFA_Bayer_Menon2007,
    mosaicing_CFA_Bayer)

In [39]:
normalized_image = image_new /1024
demosaicing_menon = demosaicing_CFA_Bayer_bilinear(normalized_image, "BGGR")
demosaicing_menon.shape

(3120, 4160, 3)

In [71]:
normalized_image[0:3,0:4]

array([[ 0.50439883,  0.20332356,  0.        ,  0.        ],
       [ 0.58064516,  0.23460411,  0.        ,  0.        ],
       [ 0.4115347 ,  0.26588465,  0.        ,  0.        ]])

In [67]:
demosaicing_menon

array([[[ 0.05859375,  0.19580078,  1.13378906],
        [ 0.1171875 ,  0.25390625,  0.37792969],
        [ 0.05859375,  0.05078125,  0.        ],
        ..., 
        [ 0.421875  ,  1.23046875,  0.24536133],
        [ 0.2109375 ,  0.24609375,  0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       [[ 0.1171875 ,  0.72509766,  0.6862793 ],
        [ 0.234375  ,  0.26220703,  0.22875977],
        [ 0.1171875 ,  0.        ,  0.        ],
        ..., 
        [ 0.84375   ,  0.74072266,  0.29248047],
        [ 0.421875  ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       [[ 0.09375   ,  0.4543457 ,  0.61669922],
        [ 0.1875    ,  0.265625  ,  0.20556641],
        [ 0.09375   ,  0.06640625,  0.        ],
        ..., 
        [ 0.625     ,  0.984375  ,  0.42138672],
        [ 0.3125    ,  0.24609375,  0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       ..., 
       [[ 0.171875  ,  1.03637695,  0.20068359],
        

In [37]:
import cv2
display_image = np.uint8(demosaicing_menon * 255)
display_image = cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR)

# Display the 8-bit image
cv2.imwrite("demosaicing_CFA_Bayer_DDFAPD_10bit.png",display_image) 

True

In [64]:
import colour_hdri #import matrix_camera_space_to_XYZ

In [65]:
from colour_hdri import matrix_camera_space_to_XYZ

ImportError: cannot import name 'matrix_camera_space_to_XYZ' from 'colour_hdri' (/Users/saisur/.local/lib/python3.8/site-packages/colour_hdri/__init__.py)